## AlekseevDP(DSU-4, DLL-15)_HW#5_Рекуррентные НС(Часть 1)_RNN1

Задание 1. Обучите нейронную сеть решать шифр Цезаря:
1. Напишите алгоритм шифра Цезаря для генерации выборки (сдвиг на К каждой
буквы. Например, при сдвиге на 2 буква “А” переходит в букву “В” и тп)
2. Сделайте нейронную сеть
3. Обучите её (вход - зашифрованная фраза, выход - дешифрованная фраза)
4. Проверьте качество

Задание 2.
Выполните практическую работу из лекционного ноутбука:
1. Постройте RNN-ячейку на основе полносвязных слоёв
2. Примените построенную ячейку для генерации текста с выражениями героев
сериала "Симпсоны

In [1]:
# Задание 1. Обучите нейронную сеть решать шифр Цезаря.
import pandas as pd
import torch
import time
import math

from random import shuffle

In [3]:
# Зададим некоторые параметры
BATCH_SIZE = 10
STRING_SIZE = 300
NUM_EPOCHS = 300
LEARNING_RATE = 0.05
FILE_NAME = "/content/onegin.txt"
CAESAR_OFFSET = 3 # смещение для шифра Цезаря

In [4]:
## выведем активное устройство для обучения модели
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print (DEVICE)

cpu


In [5]:
# Зададим класс создания алфавита из исходного файла (корпуса)
# (т.е. в алфавит будут входить ВСЕ возможные символы из корпуса, включая цифры, знаки препинания и спецсимволы)
class Alphabet(object):

    def __init__(self):
        self.letters = ""

    def __len__(self):
        return len(self.letters)

    def __contains__(self, item):
        return item in self.letters

    def __getitem__(self, item):
        if isinstance(item, int):
            return self.letters[item % len(self.letters)]
        elif isinstance(item, str):
            return self.letters.find(item)

    def __str__(self):
        letters = " ".join(self.letters)
        return f"Алфавит, сформированный из корпуса:\n {letters}\n Итого {len(self)} символа"

    def load_from_file(self, file_path):
        with open(file_path) as file:
            while True:
                text = file.read(STRING_SIZE)
                if not text:
                    break
                for ch in text:
                    if ch not in self.letters:  # если прочитанный символ отсутствует в алфавите, то добавляем его в алфавите 
                        self.letters += ch
        return self


ALPHABET = Alphabet().load_from_file(FILE_NAME)
print(ALPHABET)

Алфавит, сформированный из корпуса:
 Е в г е н и й   О 
 А л к с а д р С ч П у ш К я о ь Р м т х .   « » ( 1 8 2 3 – ) з ы п б 9 5 : Н В Г Б , э ц ж X I P t r i d e v a n l c o p u s ? ’ g q f m b - T [ щ ё ю И ф ] 	 H Х З Д ; У М ! Т Л # _ M A Ч V y — ъ Ю Э Ф … Я Ш 4 h Ц 6 Ж B k C L 7 0 O N Y E / w W Q
 Итого 134 символа


In [6]:
# класс создания датасетов 
class SentenceDataset(torch.utils.data.Dataset):

    def __init__(self, raw_data, alphabet):
        super().__init__()
        self._len = len(raw_data)
        self.y = torch.tensor(
            [[alphabet[ch] for ch in line] for line in raw_data]
        ).to(DEVICE)
        self.x = torch.tensor(
            [[i + CAESAR_OFFSET for i in line] for line in self.y]
        ).to(DEVICE)
    
    def __len__(self):
        return self._len

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

In [7]:
# функция нарезки массива исходного текста на куски размером STRING_SIZE
def get_text_array(file_path, step):
    text_array = []
    with open(file_path) as file:
        while True:
            text = file.read(STRING_SIZE)
            if not text:
                break
            text_array.append(text)
    del text_array[-1]
    return text_array

In [8]:
# заполняем трейн- и тест-выборки
raw_data = get_text_array(FILE_NAME, STRING_SIZE)
shuffle(raw_data)

_10_percent = math.ceil(len(raw_data) * 0.1)
val_data = raw_data[:_10_percent]
raw_data = raw_data[_10_percent:]

_20_percent = math.ceil(len(raw_data) * 0.2)
test_data = raw_data[:_20_percent]
train_data = raw_data[_20_percent:]

Y_val = torch.tensor([[ALPHABET[ch] for ch in line] for line in val_data])
X_val = torch.tensor([[i + CAESAR_OFFSET for i in line] for line in Y_val])

train_dl = torch.utils.data.DataLoader(
    SentenceDataset(
        train_data, ALPHABET
    ),
    batch_size=BATCH_SIZE,
    shuffle=True,
    drop_last=True
)
test_dl = torch.utils.data.DataLoader(
    SentenceDataset(
        test_data, ALPHABET
    ),
    batch_size=BATCH_SIZE,
    shuffle=True,
    drop_last=True
)

In [9]:
# класс инициализации рекуррентной НС.
# Используем размерность алфавита, загруженного из исходного корпуса (134 символа), 
# а также кол-во символов смещения шифра (CAESAR_OFFSET)
class RNNModel(torch.nn.Module):
    
    def __init__(self):
        super().__init__()
        self.embed = torch.nn.Embedding(len(ALPHABET) + CAESAR_OFFSET, 134)
        self.rnn = torch.nn.RNN(134, 134, batch_first=True)
        self.linear = torch.nn.Linear(134, len(ALPHABET) + CAESAR_OFFSET)

    def forward(self, sentence, state=None):
        embed = self.embed(sentence)
        o, h = self.rnn(embed)
        return self.linear(o)

In [10]:
# создаем модель, задаем функцию потерь и оптимизатор
model = RNNModel().to(DEVICE)
loss = torch.nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE)

In [11]:
# обучаем модель
for epoch in range(NUM_EPOCHS):
    train_loss, train_acc, iter_num = .0, .0, .0
    start_epoch_time = time.time()
    model.train()
    for x_in, y_in in train_dl:
        x_in = x_in
        y_in = y_in.view(1, -1).squeeze()
        optimizer.zero_grad()
        out = model.forward(x_in).view(-1, len(ALPHABET) + CAESAR_OFFSET)
        l = loss(out, y_in)
        train_loss += l.item()
        batch_acc = (out.argmax(dim=1) == y_in)
        train_acc += batch_acc.sum().item() / batch_acc.shape[0]
        l.backward()
        optimizer.step()
        iter_num += 1
    print(
        f"Epoch: {epoch}, loss: {train_loss:.4f}, acc: "
        f"{train_acc / iter_num:.4f}",
        end=" | "
    )
    test_loss, test_acc, iter_num = .0, .0, .0
    model.eval()
    for x_in, y_in in test_dl:
        x_in = x_in
        y_in = y_in.view(1, -1).squeeze()
        out = model.forward(x_in).view(-1, len(ALPHABET) + CAESAR_OFFSET)
        l = loss(out, y_in)
        test_loss += l.item()
        batch_acc = (out.argmax(dim=1) == y_in)
        test_acc += batch_acc.sum().item() / batch_acc.shape[0]
        iter_num += 1
    print(
        f"test loss: {test_loss:.4f}, test acc: {test_acc / iter_num:.4f} | "
        f"{time.time() - start_epoch_time:.2f} sec."
    )

Epoch: 0, loss: 42.9307, acc: 0.3520 | test loss: 7.2557, test acc: 0.6192 | 1.66 sec.
Epoch: 1, loss: 32.2231, acc: 0.7041 | test loss: 5.4864, test acc: 0.7992 | 1.06 sec.
Epoch: 2, loss: 24.8675, acc: 0.7986 | test loss: 4.3036, test acc: 0.8195 | 0.72 sec.
Epoch: 3, loss: 20.0077, acc: 0.8244 | test loss: 3.5226, test acc: 0.8535 | 0.70 sec.
Epoch: 4, loss: 16.6377, acc: 0.8516 | test loss: 2.9727, test acc: 0.8648 | 0.70 sec.
Epoch: 5, loss: 14.1686, acc: 0.8621 | test loss: 2.5365, test acc: 0.8702 | 0.71 sec.
Epoch: 6, loss: 12.4016, acc: 0.8674 | test loss: 2.2163, test acc: 0.8790 | 0.70 sec.
Epoch: 7, loss: 10.9149, acc: 0.8732 | test loss: 1.9979, test acc: 0.8842 | 0.70 sec.
Epoch: 8, loss: 9.9612, acc: 0.8817 | test loss: 1.8038, test acc: 0.8988 | 0.74 sec.
Epoch: 9, loss: 8.8690, acc: 0.8959 | test loss: 1.6813, test acc: 0.9043 | 0.71 sec.
Epoch: 10, loss: 8.3495, acc: 0.9016 | test loss: 1.5533, test acc: 0.9130 | 0.70 sec.
Epoch: 11, loss: 7.6448, acc: 0.9112 | test l

In [13]:
# Проверим на произвольном батче
idx = 5
val_results = model(X_val.to(DEVICE)).argmax(dim=2)
val_acc = (val_results == Y_val.to(DEVICE)).flatten()
val_acc = (val_acc.sum() / val_acc.shape[0]).item()
out_sentence = "".join([ALPHABET[i.item()] for i in val_results[idx]])
true_sentence = "".join([ALPHABET[i.item()] for i in Y_val[idx]])

sentence_idx = [ALPHABET[i] for i in true_sentence]
encrypted_sentence_idx = [i + CAESAR_OFFSET for i in sentence_idx]
encrypted_sentence = "".join([ALPHABET[i] for i in encrypted_sentence_idx])

print(f"Качество предсказаний : {val_acc:.4f}")
print("-" * 50)
print(f"Зашифрованный текст: \"{encrypted_sentence}\"")
print("-" * 50)
print(f"Расшифрованный текст: \"{out_sentence}\"")
print("-" * 50)
print(f"Эталонный (фактический) текст:       \"{true_sentence}\"")

Качество предсказаний : 0.9976
--------------------------------------------------
Зашифрованный текст: "эПО.кЗЗу.йаАшОр О тА ми ОА5йПйчА О.жкЗЗьПСр мПйшОн9.Аря.Ср:Пмчм.юkХА2ApvdengЧdevcpЧ?esTsп»кЗЗУСИО  ОдАнмат мр ОАОА5ПСнкЗЗ,Арф.АраяшСйАрмнрй.А й5ПСн»кккttъккЗЗц9 тА.мP мАчйат 9.Ашйамнйдм.кЗЗ	Ачя.С тАмАдПСрйА ми й
ГкЗЗьАшй.яА:йр5амч мАр5мПО тАрАнйдм.qкЗЗл:9шС
Ачйр5м А.йPАа]чй
»кЗЗ, мПм
А—СчСйнжА.м
Аен"
--------------------------------------------------
Расшифрованный текст: "Грим
		Смел чистить ногти перед ним,
		Красноречивым сумасбродом[р] (#litres_trial_promo).
		Защитник вольности и прав
		В сём случае совсем неправ.


XXV

		Быть можно дельным человеком
		И думать о красе ногтей:
		К чему бесплодно спорить с веком?
		Обычай деспот меж людей.
		Второй Чадаев, мой Ев"
--------------------------------------------------
Эталонный (фактический) текст:       "Грим
		Смел чистить ногти перед ним,
		Красноречивым сумасбродом[6] (#litres_trial_promo).
		Защитник вольности и прав
		В сём сл

In [14]:
# сделаем обратную проверку через encrypted_sentence в deencrypted_sentence, вручную подставив в sentence_to_test значение True_sentence
sentence_to_test = """Грим
		Смел чистить ногти перед ним,
		Красноречивым сумасбродом[6] (#litres_trial_promo).
		Защитник вольности и прав
		В сём случае совсем неправ.


XXV

		Быть можно дельным человеком
		И думать о красе ногтей:
		К чему бесплодно спорить с веком?
		Обычай деспот меж людей.
		Второй Чадаев, мой Ев"""
sentence_idx = [ALPHABET[i] for i in sentence_to_test]
encrypted_sentence_idx = [i + CAESAR_OFFSET for i in sentence_idx]
encrypted_sentence = "".join([ALPHABET[i] for i in encrypted_sentence_idx])
result = model(torch.tensor([encrypted_sentence_idx]).to(DEVICE)).argmax(dim=2)
deencrypted_sentence = "".join([ALPHABET[i.item()] for i in result.flatten()])
print(f"Encrypted sentence is : \"{encrypted_sentence}\"")
print("-" * 50)
print(f"De-encrypted sentence is : \"{deencrypted_sentence}\"")

Encrypted sentence is : "эПО.кЗЗу.йаАшОр О тА ми ОА5йПйчА О.жкЗЗьПСр мПйшОн9.Аря.Ср:Пмчм.юkХА2ApvdengЧdevcpЧ?esTsп»кЗЗУСИО  ОдАнмат мр ОАОА5ПСнкЗЗ,Арф.АраяшСйАрмнрй.А й5ПСн»кккttъккЗЗц9 тА.мP мАчйат 9.Ашйамнйдм.кЗЗ	Ачя.С тАмАдПСрйА ми й
ГкЗЗьАшй.яА:йр5амч мАр5мПО тАрАнйдм.qкЗЗл:9шС
Ачйр5м А.йPАа]чй
»кЗЗ, мПм
А—СчСйнжА.м
Аен"
--------------------------------------------------
De-encrypted sentence is : "Грим
		Смел чистить ногти перед ним,
		Красноречивым сумасбродом[р] (#litres_trial_promo).
		Защитник вольности и прав
		В сём случае совсем неправ.


XXV

		Быть можно дельным человеком
		И думать о красе ногтей:
		К чему бесплодно спорить с веком?
		Обычай деспот меж людей.
		Второй Чадаев, мой Ев"


## Как видим, используя для обучения архитектуру RNN, а также "расширенный" алфавит, составленный из всех символов корпуса,  удалось достичь очень высокой точности (99.76%) расшифровки текста, предварительно зашифрованного шифром Цезаря с фиксированным смещением (3 символа).

## Задание 2. Выполните практическую работу из лекционного ноутбука:

- Постройте RNN-ячейку на основе полносвязных слоёв
- Примените построенную ячейку для генерации текста с выражениями героев сериала "Симпсоны".

In [15]:
# загрузим данные
df = pd.read_csv('data.csv')
df.head()

,Unnamed: 0,id,episode_id,number,raw_text,timestamp_in_ms,speaking_line,character_id,location_id,raw_character_text,raw_location_text,spoken_words,normalized_text,word_count
0,0,10368,35,29,"Lisa Simpson: Maggie, look. What's that?",235000,True,9,5.0,Lisa Simpson,Simpson Home,"Maggie, look. What's that?",maggie look whats that,4.0
1,1,10369,35,30,Lisa Simpson: Lee-mur. Lee-mur.,237000,True,9,5.0,Lisa Simpson,Simpson Home,Lee-mur. Lee-mur.,lee-mur lee-mur,2.0
2,2,10370,35,31,Lisa Simpson: Zee-boo. Zee-boo.,239000,True,9,5.0,Lisa Simpson,Simpson Home,Zee-boo. Zee-boo.,zee-boo zee-boo,2.0
3,3,10372,35,33,Lisa Simpson: I'm trying to teach Maggie that ...,245000,True,9,5.0,Lisa Simpson,Simpson Home,I'm trying to teach Maggie that nature doesn't...,im trying to teach maggie that nature doesnt e...,24.0
4,4,10374,35,35,"Lisa Simpson: It's like an ox, only it has a h...",254000,True,9,5.0,Lisa Simpson,Simpson Home,"It's like an ox, only it has a hump and a dewl...",its like an ox only it has a hump and a dewlap...,18.0


In [16]:
phrases = df['normalized_text'].tolist()
phrases[:10]

['maggie look whats that',
 'lee-mur lee-mur',
 'zee-boo zee-boo',
 'im trying to teach maggie that nature doesnt end with the barnyard i want her to have all the advantages that i didnt have',
 'its like an ox only it has a hump and a dewlap hump and dew-lap hump and dew-lap',
 'you know his blood type how romantic',
 'oh yeah whats my shoe size',
 'ring',
 'yes dad',
 'ooh look maggie what is that do-dec-ah-edron dodecahedron']

In [17]:
text = [[c for c in ph] for ph in phrases if type(ph) is str]

## Делаем массив с данными

In [18]:
CHARS = set('abcdefghijklmnopqrstuvwxyz ')
INDEX_TO_CHAR = ['none'] + [w for w in CHARS]
CHAR_TO_INDEX = {w: i for i, w in enumerate(INDEX_TO_CHAR)}

In [19]:
MAX_LEN = 50
X = torch.zeros((len(text), MAX_LEN), dtype=int)
for i in range(len(text)):
    for j, w in enumerate(text[i]):
        if j >= MAX_LEN:
            break
        X[i, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])

X.to(DEVICE)

tensor([[20, 16,  5,  ...,  0,  0,  0],
        [12,  3,  3,  ...,  0,  0,  0],
        [25,  3,  3,  ...,  0,  0,  0],
        ...,
        [11, 10, 21,  ...,  0,  0,  0],
        [20, 16,  7,  ..., 27, 12,  3],
        [16, 11, 27,  ...,  0,  0,  0]])

In [20]:
X[0:3]

tensor([[20, 16,  5,  5,  2,  3, 27, 12, 10, 10, 23, 27, 21, 11, 16, 14, 26, 27,
         14, 11, 16, 14,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [12,  3,  3,  0, 20,  8, 24, 27, 12,  3,  3,  0, 20,  8, 24,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [25,  3,  3,  0,  9, 10, 10, 27, 25,  3,  3,  0,  9, 10, 10,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]])

## Смотрим на Embedding и RNN ячейку

In [21]:
embeddings = torch.nn.Embedding(len(INDEX_TO_CHAR), 28)
t = embeddings(X[0:3])
t

tensor([[[ 0.7476,  1.8638, -0.5268,  ...,  0.6672, -0.7020,  0.2717],
         [ 1.6652,  0.2178, -0.2961,  ..., -1.4403,  1.1554, -0.8749],
         [ 1.7135, -1.8144,  0.3340,  ...,  0.9383, -0.3564, -0.5927],
         ...,
         [ 0.7903, -0.3701,  1.7428,  ...,  0.2809, -0.0804, -1.6894],
         [ 0.7903, -0.3701,  1.7428,  ...,  0.2809, -0.0804, -1.6894],
         [ 0.7903, -0.3701,  1.7428,  ...,  0.2809, -0.0804, -1.6894]],

        [[-0.5318,  0.6047, -0.9767,  ...,  2.1632, -1.0145, -0.2269],
         [ 0.3482, -1.0890,  1.3401,  ...,  0.0245, -0.5866, -0.9971],
         [ 0.3482, -1.0890,  1.3401,  ...,  0.0245, -0.5866, -0.9971],
         ...,
         [ 0.7903, -0.3701,  1.7428,  ...,  0.2809, -0.0804, -1.6894],
         [ 0.7903, -0.3701,  1.7428,  ...,  0.2809, -0.0804, -1.6894],
         [ 0.7903, -0.3701,  1.7428,  ...,  0.2809, -0.0804, -1.6894]],

        [[-0.7473,  0.7941,  0.6742,  ...,  0.5950, -0.6790,  0.8056],
         [ 0.3482, -1.0890,  1.3401,  ...,  0

In [22]:
t.shape, X[0:3].shape

(torch.Size([3, 50, 28]), torch.Size([3, 50]))

In [23]:
rnn = torch.nn.RNN(28, 128, batch_first=True)
o, s = rnn(t)
o.shape, s.shape

(torch.Size([3, 50, 128]), torch.Size([1, 3, 128]))

In [24]:
o, s2 = rnn(t, s)
o.shape, s2.shape

(torch.Size([3, 50, 128]), torch.Size([1, 3, 128]))

In [25]:
# 3 слоя - embedding (28), скрытая ячейка (128), полносвязанный из состояния rnn в букву (28)
class Network(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.embed = torch.nn.Embedding(28, 128)
        self.rnn = torch.nn.RNN(128, 128, batch_first=True)
        self.linear = torch.nn.Linear(128, 28)
        
    def forward(self, sentence, state=None):
        embed = self.embed(sentence)
        o, h = self.rnn(embed)
        return self.linear(o)

In [33]:
NUM_EPOCHS = 200
LEARNING_RATE = 0.01

In [27]:
## выведем активное устройство для обучения модели
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print (DEVICE)

cpu


In [34]:
model = Network().to(DEVICE)

In [36]:
criterion = torch.nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

## Практика. Реализуйте код генерации следующей буквы на основе модели
Логика такая:
    - Сначала кормим в нее буквы из sentence (прогревая состояние)
    - Затем пока не получим none (0) берем самую вероятную букву и добавляем ее в sentence
    - Повторяем

In [37]:
def generate_sentence():
    sentence = ['h', 'e', 'l', 'l', 'o']
    x = torch.zeros((1, len(sentence)), dtype=int)
    
    for j, w in enumerate(sentence):
        if j >= MAX_LEN:
            break
        
        else:
            x[0, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])
    
    for i in range(MAX_LEN):
        o = model(x)
        w = torch.argmax(o[-1, -1, :], keepdim=True)
        x = torch.cat([x, w.unsqueeze(0)], axis=1)
        ww = INDEX_TO_CHAR[w]
        
        if ww == 'none':
            break
            
        sentence.append(ww)
    
    print(' '.join(sentence))

In [38]:
generate_sentence()

h e l l o


In [39]:
for ep in range(NUM_EPOCHS):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    for i in range(int(len(X) / 100)):
        batch = X[i * 100:(i + 1) * 100]
        X_batch = batch[:, :-1]
        Y_batch = batch[:, 1:].flatten()

        optimizer.zero_grad()
        answers = model.forward(X_batch)
        answers = answers.view(-1, len(INDEX_TO_CHAR))
        loss = criterion(answers, Y_batch)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

    print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))
    generate_sentence()

Epoch 0. Time: 5.508, Train loss: 1.390
h e l l o   t h e   s e e   s t a r t   t h e   s e e   s t a r t   t h e   s e e   s t a r t   t h e   s e e
Epoch 1. Time: 5.441, Train loss: 1.179
h e l l o w   a   s a t e   a   s a t e   a   s a t e   a   s a t e   a   s a t e   a   s a t e   a   s a t e
Epoch 2. Time: 5.533, Train loss: 1.130
h e l l o   t h e   s e e   t h e   s e e   t h e   s e e   t h e   s e e   t h e   s e e   t h e   s e e   t
Epoch 3. Time: 5.519, Train loss: 1.105
h e l l o   t h e   s e e   t h e   s e e   t h e   s e e   t h e   s e e   t h e   s e e   t h e   s e e   t
Epoch 4. Time: 5.536, Train loss: 1.089
h e l l o   t h e   s p e a t   t h e   s p e a t   t h e   s p e a t   t h e   s p e a t   t h e   s p e a t
Epoch 5. Time: 5.556, Train loss: 1.078
h e l l o   i t   s t a r t   t h e   s c h o o l   m a k e   t o   s e e   y o u   s e e   t h e   s c h o o
Epoch 6. Time: 5.546, Train loss: 1.069
h e l l o   i t s   a   l i t t l e   t o   s e e m   t h e 

## Как видим, обучение происходит достаточно медленно и затейливо... Тем не менее, замена оптимизатора с SGD на AdamW и уменьшение LR с 0.05 до 0.01 привели к улучшению процесса обучения RNN.